# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import nltk
import re
from nltk.tokenize import word_tokenize
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# nltk.download('punkt')

In [3]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_df.db')
df = pd.read_sql_table('disaster_messages', engine)
X = df['message']
Y = df.loc[:,'related':'direct_report']

### 2. Write a tokenization function to process your text data

In [4]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [5]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]",' ', text)
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_token = []
    for token in tokens:
        text_token = lemmatizer.lemmatize(token).lower().strip()
        clean_token.append(text_token)
    
    return clean_token

In [6]:
X[155]

'the people in the village of god need food and water so that they can stay. It is isn the university of Kiskeya ( in the area of delams 35 right of the main delmas road ). pleas respond'

In [7]:
tokenize(X[155])

['the',
 'people',
 'in',
 'the',
 'village',
 'of',
 'god',
 'need',
 'food',
 'and',
 'water',
 'so',
 'that',
 'they',
 'can',
 'stay',
 'it',
 'is',
 'isn',
 'the',
 'university',
 'of',
 'kiskeya',
 'in',
 'the',
 'area',
 'of',
 'delams',
 '35',
 'right',
 'of',
 'the',
 'main',
 'delmas',
 'road',
 'plea',
 'respond']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
np.array(y_test)

array([['0', '0', '0', ..., '0', '0', '0'],
       ['1', '0', '0', ..., '0', '0', '0'],
       ['0', '0', '0', ..., '0', '0', '0'],
       ...,
       ['0', '0', '0', ..., '0', '0', '0'],
       ['1', '0', '0', ..., '0', '0', '0'],
       ['0', '0', '0', ..., '0', '0', '0']], dtype=object)

In [11]:
target_names = list(Y.columns)

In [12]:
y_pred_df = pd.DataFrame(y_pred, columns=target_names)

In [13]:
for i in y_pred_df.columns:
    print(i, '\n',classification_report(y_test[i], y_pred_df[i]))

related 
              precision    recall  f1-score   support

          0       0.65      0.36      0.46      1568
          1       0.82      0.94      0.87      4930
          2       0.86      0.38      0.53        47

avg / total       0.78      0.80      0.77      6545

request 
              precision    recall  f1-score   support

          0       0.88      0.99      0.93      5411
          1       0.85      0.38      0.52      1134

avg / total       0.88      0.88      0.86      6545

offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6513
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6545



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


aid_related 
              precision    recall  f1-score   support

          0       0.71      0.88      0.79      3825
          1       0.76      0.50      0.60      2720

avg / total       0.73      0.73      0.71      6545

medical_help 
              precision    recall  f1-score   support

          0       0.93      1.00      0.96      6024
          1       0.67      0.07      0.13       521

avg / total       0.90      0.92      0.89      6545

medical_products 
              precision    recall  f1-score   support

          0       0.96      1.00      0.98      6234
          1       0.68      0.08      0.14       311

avg / total       0.94      0.95      0.94      6545

search_and_rescue 
              precision    recall  f1-score   support

          0       0.97      1.00      0.99      6360
          1       0.67      0.02      0.04       185

avg / total       0.96      0.97      0.96      6545

security 
              precision    recall  f1-score   support

       

### 6. Improve your model
Use grid search to find better parameters. 

In [27]:
parameters = {
#     'tfidf__use_idf': (True, False),
#     'tfidf__stop_words':(None, 'english'),
#     'tfidf__ngram_range': ((1, 1),(1,3)),
    'clf__estimator__n_estimators': (15, 20),
#     'clf__estimator__min_samples_split': (2, 4),
#     'clf__estimator__max_features': ('auto', .5)
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [30]:
y_pred_tuned = cv.predict(X_test)

In [31]:
y_pred_df_tuned = pd.DataFrame(y_pred_tuned, columns=target_names)

In [34]:
for i in y_pred_df_tuned.columns:
    print(i, '\n',classification_report(y_test[i], y_pred_df_tuned[i]))

related 
              precision    recall  f1-score   support

          0       0.73      0.34      0.46      1568
          1       0.82      0.96      0.88      4930
          2       0.55      0.34      0.42        47

avg / total       0.79      0.81      0.78      6545

request 
              precision    recall  f1-score   support

          0       0.88      0.99      0.93      5411
          1       0.88      0.38      0.53      1134

avg / total       0.88      0.88      0.86      6545

offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6513
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6545



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


aid_related 
              precision    recall  f1-score   support

          0       0.74      0.88      0.81      3825
          1       0.77      0.57      0.65      2720

avg / total       0.75      0.75      0.74      6545

medical_help 
              precision    recall  f1-score   support

          0       0.92      1.00      0.96      6024
          1       0.61      0.05      0.10       521

avg / total       0.90      0.92      0.89      6545

medical_products 
              precision    recall  f1-score   support

          0       0.96      1.00      0.98      6234
          1       0.72      0.14      0.23       311

avg / total       0.95      0.96      0.94      6545

search_and_rescue 
              precision    recall  f1-score   support

          0       0.97      1.00      0.99      6360
          1       1.00      0.02      0.04       185

avg / total       0.97      0.97      0.96      6545

security 
              precision    recall  f1-score   support

       

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [37]:
filename = 'classifier.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.